# Presentation of project for Schlumberger interview

## Solving Euler equations for compressible gas in 2 dimensions

**Problem set in Math228b, Numerical solutions of differential equations, at University of California, Berkeley**

Simulate a flow of gas in two dimensions in square domain with periodic boundary conditions. Solved in Julia.



$
\Huge
\partial_t u = - \nabla F 
$

- $u$: state vector with 4 components. Has one value for each point in the domain, for each component.
- $F$: Fluxes
- $\nabla F$: divergence

$
\
u=\begin{bmatrix} \rho \\ \rho u \\ \rho v \\ \rho E \end{bmatrix} 
\hspace{2mm} \text{and} \hspace{3mm} 
F = \begin{bmatrix} \rho u & \rho v \\ \rho u^2 + p & \rho u v \\ \rho u v & \rho v^2 + p \\ u(\rho E + p) & v(\rho E + p) \end{bmatrix} 
\hspace{2mm} \text{and} \hspace{3mm} 
p = (\gamma - 1)\rho(E - (u^2+v^2)/2)
$

- $\rho$: fluid density
- $u, v$: fluid velocity components
- $E$: total energy
- $p$: pressure
- $\gamma$: adiabatic gas constant

In [4]:
#! Starting with imports
using LinearAlgebra, SparseArrays
using ProgressMeter
using Plots
using LaTeXStrings
Plots.pyplot()

Plots.PyPlotBackend()

In [6]:
pressure(r, ru, rv, rE, γ=1.4) = @. (γ - 1) * (rE - (ru^2 + rv^2)/2r)

function euler_fluxes(r, ru, rv, rE)
    """
    Return the x any y components of the fluxes of the solution components
    Matrix F in (4) in PS2

    Arguments:
        r, ru, rv, rE: Solution components
    Returns:
        Frx, Fry, Frux, Fruy, Frvx, Frvy, FrEx, FrEy: flux-componets
    """
    p = pressure(r, ru, rv, rE)
    Frx  = ru
    Fry  = rv
    Frux = @. ru^2 / r + p
    Fruy = @. ru * rv / r
    Frvx = @. ru * rv / r
    Frvy = @. rv^2 / r + p
    FrEx = @. ru / r * (rE + p)
    FrEy = @. rv / r * (rE + p)
    return Frx, Fry, Frux, Fruy, Frvx, Frvy, FrEx, FrEy
end

function euler_rhs(r, ru, rv, rE, h, mats)
    """
    Rhs of differential equation du_dt = -divF(u)
    Calculates fluxes of each component,
    calculates divergence of each component using fluxes
    returns divergence of solution
    """
    Frx, Fry, Frux, Fruy, Frvx, Frvy, FrEx, FrEy = euler_fluxes(r, ru, rv, rE)
    fr  = compact_div(Frx , Fry , h, mats)
    fru = compact_div(Frux, Fruy, h, mats)
    frv = compact_div(Frvx, Frvy, h, mats)
    frE = compact_div(FrEx, FrEy, h, mats)
    return -fr, -fru, -frv, -frE
end

euler_rhs (generic function with 1 method)

Partial differential equation (PDE), with derivatives in space and time. Will solve it with finite difference methods. The spatial derivates are discretized with the fourth-order compact Padé scheme. The time dimentions is solved by RungeKutta4. 

1-dimensional Padé scheme is 

$ f'_{i-1} + 4f'_i + f'_{i+1} = 3 (f_{i+1} - f_{i-1}) / h$,

where $h$ is the step length of the discretization.

In [1]:
function make_pade_mats(n)
    """
    Pre-compute the matrices used to solve spatial derivates in x-direction.
    
    Arguments:
        n: number of points in discretisation
    Returns:
        mats:
            mats[1] = lhs of differentiation
            mats[2] = rhs of differentiation
    """
    #* Sparse matrices, tri-diagonal, but with periodic boundaries
    div_lhs = spdiagm(-n+1=>[1], -1=>ones(n-1), 0=>4ones(n), 1=>ones(n-1), n-1=>[1])
    div_rhs = spdiagm(-n+1=>[1], -1=>-ones(n-1), 1=>ones(n-1), n-1=>[-1])

    #* pre-compute LU-factorisation, to significantly speed-up (~6x) solving by backslash
    LU = lu(div_lhs)
    return [LU, div_rhs]
end

make_pade_mats (generic function with 1 method)

In [2]:
function pade_x(f, h, mats)
    """
    Calculates derivative in x-direction (lines) of nxn square matrix

    Arguments:
        f: square matrix to calculate line-derivaties of
        h: spatial steplength
        mats: pre-generated matricies performing
              derivative according to equation in part b
    Returns:
        df: derivative of f
    """
    lhs, rhs = mats
    srhs = rhs * 3 / h  # scale with correct params

    df = lhs \ (srhs * f)
    return df
end
function compact_div(Fx, Fy, h, mats)
    """
    Calculates divergence of vector-field F = (Fx, Fy)
    This is the sum of the derivative in x-direction and y-direction:
        divF = dFx + dFy

    Arguments:
        Fx, Fy: x and y part of vector-field F to compute flux of
        h: spatial steplength
        mats: pre-computed matrices, see doc of pade_x
    Returns:
        divF: divergence of field F
    """
    dFx = pade_x(Fx, h, mats)
    dFy = pade_x(Matrix(Fy'), h, mats)
    return dFx .+ Matrix(dFy')
end

compact_div (generic function with 1 method)

Due to the nature of the problem and the pade-scheme, the amplitude of high-frequency waves in the spatial solution will increase and diverge. Therefore, the solution will be filtered at every time step, eliminating these waves. It is a 6th order compact filter, parameterised $\alpha$:

$
\hat{f}_i + \alpha (\hat{f}_{i-1} + \hat{f}_{i-1}) = af_i + \frac{b}{2}(f_{i-1} + f_{i+1}) + \frac{c}{2}(f_{i-2} + f_{i+2}) 
$

 where $a = 5/8 + 3\alpha/4$ and $b = \alpha + 1/2$ and $c = \alpha/4 + 1/8$.

 $\alpha=0.5$ means no filtering, $\alpha < 0.5$ means low-pass filtering, while $\alpha > 0.5$ means high-pass filtering. 

In [7]:
function make_filter_mats(n, α)
    """
    Pre-compute filter matricies

    Arguments:
        n: number of points in discretisation
        α: filtering-parameter
    Returns:
        mats:
            mats[1] = lhs of filtering
            mats[2] = rhs of filtering
    """
    # filter-params
    a = (5. / 8 + 3. * α / 4) * ones(n)
    b(n) = (α + 0.5) * ones(n) / 2
    c(n) = (α / 4. - 1/8) * ones(n) / 2

    filter_rhs = spdiagm(-n+1=>b(1), -n+2=>c(2), -2=>c(n-2), -1=>b(n-1), 0=>a, 1=>b(n-1), 2=>c(n-2), n-2=>c(2), n-1=>b(1))
    filter_lhs = spdiagm(-n+1=>[α], -1=>α*ones(n-1), 0=>ones(n), 1=>α*ones(n-1), n-1=>[α])

    # pre-compute LU-factorisation, to significantly speed-up (~6x) solving by backslash
    LU = lu(filter_lhs)

    return [LU, filter_rhs]
end

function make_matrices(n, α)
    #* Combine the matrix-maker functions
    return [make_pade_mats(n), make_filter_mats(n, α)]
end

make_matrices (generic function with 1 method)

In [8]:
function filter_x(u, filters)
    """
    Filters square matrix u in x-direction

    Arguments:
        u: square matrix to filter in x-dir
        filters: pre-computed matrices, same method as for pade_x
    Returns:
        f: filtered u
    """
    lhs, rhs = filters

    f = lhs \ (rhs * u)
    return f
end

function compact_filter(u, filters)
    """
    Filters square matrix u in x and y direction

    Arguments:
        u: square matrix to filter
        filters: pre-computed filter matricies, see doc of filter_x
    Returns:
        fu: filtered u
    """
    u = filter_x(u, filters)  # filter in x-direction
    u = filter_x(Matrix(u'), filters) # filter in y-direction
    return Matrix(u')
end

compact_filter (generic function with 1 method)

Now time to solve time-part. Uses RK4 and the "method of lines".

In [9]:
function rk4step(r, ru, rv, rE, h, k, mats)
    """
    Takes a single rk4-step of solution
    filters solution before returning

    Arguments
        r, ru, rv, rE: components of solution, each element is a nxn matrix
        h: spatial stepsize
        k: temporal steplength
        mats: pre-computed matricies for derivating and filtering
    Returns:
        Next step of r, ru, rv, rE
    """
    pade, filters = mats  #* divergence and filter matricies
    k = 0.5 * k           #* temporal stepsize, halved for convenience
    #* standard sub-steps of rk4
    r1, ru1, rv1, rE1 = euler_rhs(r, ru, rv, rE, h, pade)
    r2, ru2, rv2, rE2 = euler_rhs(r+ k*r1, ru+ k*ru1, rv+ k*rv1, rE+ k*rE1, h, pade)
    r3, ru3, rv3, rE3 = euler_rhs(r+ k*r2, ru+ k*ru2, rv+ k*rv2, rE+ k*rE2, h, pade)
    r4, ru4, rv4, rE4 = euler_rhs(r+2k*r3, ru+2k*ru3, rv+2k*rv3, rE+2k*rE3, h, pade)

    #* update each component
    r  += k/3 * (r1  + 2r2  + 2r3  + r4 )
    ru += k/3 * (ru1 + 2ru2 + 2ru3 + ru4)
    rv += k/3 * (rv1 + 2rv2 + 2rv3 + rv4)
    rE += k/3 * (rE1 + 2rE2 + 2rE3 + rE4)

    #* filter each component
    r  = compact_filter(r , filters)
    ru = compact_filter(ru, filters)
    rv = compact_filter(rv, filters)
    rE = compact_filter(rE, filters)

    return r, ru, rv, rE
end

rk4step (generic function with 1 method)

Now all steps are in place, just need something to solve. Will first verify implementation on simple system, a 2d gaussian point, for which we have the analytic solution. Then I will simulate a system where one flow region has higher density and speed than the neighboring region.